In [4]:
from radiosim.modems import QPSKModem
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'radiosim'

In [42]:
qpsk = QPSKModem(sps=4)

In [43]:
data = np.random.rand(20)*100
data = data.astype(np.uint8)

In [57]:
symbols = qpsk.map(data)

rcc = qpsk.get_pulse_filter(sps=4)
bb = qpsk.apply_pulse_filter(symbols, rcc)

bits = np.unpackbits(data)
interp_bits = np.zeros(len(bits)*4, dtype=np.uint8)
interp_bits[::4] = bits

In [58]:
plt.stem(interp_bits)
plt.plot(bb.real)

/tmp/ipykernel_22997/3457396072.py:1: UserWarning: In Matplotlib 3.3 individual lines on a stem plot will be added as a LineCollection instead of individual lines. This significantly improves the performance of a stem plot. To remove this warning and switch to the new behaviour, set the "use_line_collection" keyword argument to True.
  plt.stem(interp_bits)
